<a href="https://colab.research.google.com/github/mdsaifhaider/kaggle-digit-recognizer-challenge/blob/main/Kaggle1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [431]:
###kaggle score of 0.94264

In [430]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras import layers

In [323]:
import csv
import zipfile 
import os

In [324]:
local_file="/digit-recognizer.zip"
local_ref=zipfile.ZipFile(local_file,"r")
local_ref.extractall("/digit-recognizer")
local_ref.close()

In [325]:
train_file=pd.read_csv("/digit-recognizer/train.csv")
test_file=pd.read_csv("/digit-recognizer/test.csv")

In [326]:
train_file.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [327]:
test_file

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
27996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,183,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,51,252,20,0,0,0,0,0,0,0,0,0,0,0
27997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
27998,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [397]:
train_labels=[]
train_images=[]
with open("/digit-recognizer/train.csv","r") as file:
      reader=csv.reader(file)
      next(reader,None)
      for row in reader:
            label=np.array(row[0]).astype(int)
            image=np.array(row[1:785]).astype(int)
            image=image.reshape((28,28))
            train_labels.append(label)
            train_images.append(image)

train_images=np.array(train_images).astype(int)
train_labels=np.array(train_labels).astype(int)
train_images=np.expand_dims(train_images,axis=3)

In [398]:
print(len(train_images))
print(len(train_labels))

42000
42000


In [399]:
test_images=[]
with open("/digit-recognizer/test.csv","r") as file:
    reader=csv.reader(file)
    next(reader,None)
    for row in reader:
      image=np.array(row[0:784]).astype(int)
      image=image.reshape((28,28))
      test_images.append(image)
test_images=np.array(test_images).astype(int)
test_images=np.expand_dims(test_images,axis=3)

In [400]:
len(test_images)

28000

In [ ]:

model=tf.keras.models.Sequential([
                       tf.keras.layers.Conv2D(32,(3,3),kernel_initializer="he_normal",activation="relu",input_shape=(28,28,1)),
                       tf.keras.layers.Conv2D(32,(3,3),kernel_initializer="he_normal",activation="relu"),
                       tf.keras.layers.MaxPooling2D(2,2),
      
                       tf.keras.layers.Conv2D(64,(3,3),activation="relu",padding="same"),
                       tf.keras.layers.MaxPooling2D(2,2),
                       tf.keras.layers.Conv2D(64,(3,3),activation="relu",padding="same"),
                       tf.keras.layers.MaxPooling2D(2,2),
                       tf.keras.layers.Conv2D(64,(3,3),activation="relu",padding="same"),
                       tf.keras.layers.MaxPooling2D(2,2),
                   
                       tf.keras.layers.Dropout(0.1),
                       tf.keras.layers.Flatten(),
                       tf.keras.layers.Dense(512,activation="relu"),
                       tf.keras.layers.Dense(64,activation="relu"),
                       tf.keras.layers.Dense(10,activation="softmax")   
                    ])


In [ ]:
model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=["accuracy"])

In [403]:
val_images=train_images[40000:,:,:]
val_images.shape

(2000, 28, 28, 1)

In [404]:
train_images=train_images[:40000,:,:]
train_images.shape

(40000, 28, 28, 1)

In [405]:
test_images.shape

(28000, 28, 28, 1)

In [406]:
val_labels=train_labels[40000:]
val_labels.shape

(2000,)

In [407]:
train_labels=train_labels[:40000]
train_labels.shape

(40000,)

In [408]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [409]:
train_data_gen=ImageDataGenerator(rescale=1.0/255,
                                  rotation_range=40,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  shear_range=0.1,
                                  horizontal_flip=True,
                                  zoom_range=0.2,
                                  fill_mode="nearest"
                                  )

In [410]:
train_gen=train_data_gen.flow(
    train_images,
    train_labels,
    batch_size=64
   
)

In [411]:
val_data_gen=ImageDataGenerator(rescale=1.0/255)

In [412]:
val_gen=val_data_gen.flow(val_images,val_labels,
                          batch_size=64,
                          )

In [413]:
test_data_gen=ImageDataGenerator(rescale=1.0/255)

In [420]:
history=model.fit(train_gen,epochs=30,validation_data=val_gen)

Epoch 1/30
625/625 [==============================] - 79s 127ms/step - loss: 0.1596 - accuracy: 0.9522 - val_loss: 0.1083 - val_accuracy: 0.9675
Epoch 2/30
625/625 [==============================] - 78s 125ms/step - loss: 0.1544 - accuracy: 0.9526 - val_loss: 0.0835 - val_accuracy: 0.9740
Epoch 3/30
625/625 [==============================] - 81s 129ms/step - loss: 0.1464 - accuracy: 0.9562 - val_loss: 0.1032 - val_accuracy: 0.9635
Epoch 4/30
625/625 [==============================] - 79s 127ms/step - loss: 0.1395 - accuracy: 0.9586 - val_loss: 0.0812 - val_accuracy: 0.9725
Epoch 5/30
625/625 [==============================] - 78s 125ms/step - loss: 0.1304 - accuracy: 0.9602 - val_loss: 0.0970 - val_accuracy: 0.9695
Epoch 6/30
625/625 [==============================] - 78s 125ms/step - loss: 0.1295 - accuracy: 0.9614 - val_loss: 0.0744 - val_accuracy: 0.9765
Epoch 7/30
625/625 [==============================] - 80s 128ms/step - loss: 0.1278 - accuracy: 0.9614 - val_loss: 0.0844 - val_ac

In [421]:
prediction=model.predict(test_images)

In [422]:
prediction=np.round(prediction)

In [423]:
df=pd.DataFrame(prediction)

In [424]:
df

,0,1,2,3,4,5,6,7,8,9
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
27995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
27996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
27997,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
27998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [425]:
Label=[]
ImageId=[]
for i in range(len(prediction)):
  for j in range(0,10):
    if(prediction[i][j]==1):
        Label.append(j)
        ImageId.append(i+1)

In [426]:
df=pd.DataFrame(ImageId,columns=["ImageId"])

In [427]:
df

,ImageId
0,1
1,2
2,3
3,4
4,5
...,...
27995,27996
27996,27997
27997,27998
27998,27999


In [428]:
final_df=df.assign(Label=Label)
final_df

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9


In [429]:
final_df.to_csv("submission.csv")

# New Section